## Step 1 - Data Preprocessing

In [2]:
import pandas as pd
import glob
import os

### Gather RNA counts

In [3]:
path = "Data"
filenames = glob.glob(path + "/*.txt")

# Create the dataframes for each file
dfs = []
for filename in filenames:
    df = pd.read_csv(filename, sep="\t", names=[filename], skiprows=1).T
    dfs.append(df)

# Concatenate all the datadrames into one
rna_counts = pd.concat(dfs)
rna_counts

,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2MP1,A3GALT2,A4GALT,A4GNT,...,X6B_LINE:CR1:LINE,X7A_LINE:CR1:LINE,X7B_LINE:CR1:LINE,X7C_LINE:CR1:LINE,X7D_LINE:CR1:LINE,X8_LINE:CR1:LINE,X9_LINE:L1:LINE,Zaphod2:hAT-Tip100:DNA,Zaphod3:hAT-Tip100:DNA,Zaphod:hAT-Tip100:DNA
Data/GSM3533247_CGND-HRA-00059_counts.txt,23,100,0,1641,75,128,2,1,21,0,...,855,2046,1843,1161,84,492,142,454,1602,1665
Data/GSM3533313_CGND-HRA-00354_counts.txt,27,108,0,1609,89,110,5,1,12,0,...,755,1562,1570,782,95,459,123,360,1273,1560
Data/GSM3533338_CGND-HRA-00423_counts.txt,28,100,1,1025,78,105,2,1,26,0,...,580,1216,1192,688,65,299,108,274,1081,1143
Data/GSM3533281_CGND-HRA-00241_counts.txt,16,65,0,2425,51,123,2,0,113,0,...,570,1389,1210,689,71,364,81,296,1201,1422
Data/GSM3533358_CGND-HRA-00561_counts.txt,55,164,1,1620,138,85,4,8,42,0,...,666,1590,1465,823,79,380,85,325,1262,1465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Data/GSM3533300_CGND-HRA-00300_counts.txt,24,97,0,3752,141,365,2,5,97,0,...,866,1995,1791,1053,85,501,108,427,1732,1730
Data/GSM3533233_CGND-HRA-00016_counts.txt,22,101,0,2131,142,143,5,1,36,0,...,935,2278,2032,1198,97,562,132,540,2032,2135
Data/GSM3533359_CGND-HRA-00562_counts.txt,29,145,0,1424,119,111,1,4,52,0,...,751,1853,1655,922,89,503,131,463,1624,1742
Data/GSM3533310_CGND-HRA-00338_counts.txt,29,88,0,1171,86,105,3,2,37,1,...,809,1790,1590,764,93,435,125,350,1579,1871


## Gather sample annotations

In [4]:
filename = "Data/GSE124439_family.xml"
annotations = pd.read_xml(filename)

annotations

,iid,Person,Email,Laboratory,Department,Organization,Address,Name,Public-ID,Web-Link,...,Library-Source,Library-Selection,Instrument-Model,Supplementary-Data,Relation,Pubmed-ID,Summary,Overall-Design,Contributor-Ref,Sample-Ref
0,contrib1,NaN,dfagegaltier@nygenome.org,Hemali Phatnani,Center for Genomics of Neurodegenerative Disease,New York Genome Center,NaN,None,None,None,...,None,None,NaN,None,NaN,NaN,None,None,NaN,NaN
1,contrib2,NaN,None,None,None,GEO,NaN,None,None,None,...,None,None,NaN,None,NaN,NaN,None,None,NaN,NaN
2,contrib3,NaN,None,None,None,None,NaN,None,None,None,...,None,None,NaN,None,NaN,NaN,None,None,NaN,NaN
3,contrib4,NaN,None,None,None,None,NaN,None,None,None,...,None,None,NaN,None,NaN,NaN,None,None,NaN,NaN
4,contrib5,NaN,None,None,None,None,NaN,None,None,None,...,None,None,NaN,None,NaN,NaN,None,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,GSM3533402,NaN,None,None,None,None,NaN,None,None,None,...,transcriptomic,cDNA,NaN,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3533...,NaN,NaN,None,None,NaN,NaN
180,GSM3533403,NaN,None,None,None,None,NaN,None,None,None,...,transcriptomic,cDNA,NaN,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3533...,NaN,NaN,None,None,NaN,NaN
181,GSM3533404,NaN,None,None,None,None,NaN,None,None,None,...,transcriptomic,cDNA,NaN,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3533...,NaN,NaN,None,None,NaN,NaN
182,GSM3533405,NaN,None,None,None,None,NaN,None,None,None,...,transcriptomic,cDNA,NaN,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3533...,NaN,NaN,None,None,NaN,NaN


In [1]:
filename = "sample.xml"
annotations = pd.read_xml(filename, xpath="./Sample", namespaces="http://www.ncbi.nlm.nih.gov/geo/info/MINiML")

annotations

NameError: name 'pd' is not defined